In [1]:
import numpy as np
import csv
import pandas as pd
import math

# Euclidean Distance Correlation Matrix

Generate a 53x53 matrix that stores the distances between each state for correlation
Generate a 1x53 vector containing the widths of the kernel based on the k closest states
Generate a 53x53 matrix representing the correlation between any two states

In [2]:
def gen_distances(state_demographics):
    dist_array = np.empty([53,53])
    for i in range(len(state_demographics)):
        for j in range(len(state_demographics)):
            dist_array[i,j]= np.linalg.norm(np.array(state_demographics[i])-np.array(state_demographics[j]))
    return dist_array

In [3]:
def gamma(dist_array,k):
    gamma = []
    for state in dist_array:
        k_shortest = np.argpartition(state, k)
        gamma.append(-1/(2*(np.linalg.norm(state[k_shortest[1:k+1]])))**2)    
    return gamma

In [4]:
def rbf_kernel(dist_array, gamma):
    correlation_matrix = np.empty([53,53])
    for i in range(len(dist_array)):
        for j in range(len(dist_array)):
            if i == j:
                correlation_matrix[i,j] = 0
            else:
                if gamma[i] == -math.inf:
                    correlation_matrix[i,j] = 0
                else:
                    correlation_matrix[i,j] = np.exp(gamma[i]*(dist_array[i,j]**2))
    return correlation_matrix


# Seeding State Shifts, but BETTER!
generates arbitrary number of seeds

In [5]:
def run_seeding_state_shifts(state_info, correlation_matrix, num_simulations, num_seeds):

    electoral_college_outcomes = [0 for x in range(539)]
    for sim in range(num_simulations):
        seed = []
        for i in range(num_seeds):
            seed.append(np.random.randint(53))

        seeded_value = [(np.random.normal(state_info[seed[x]][1],(state_info[seed[x]][2])**0.5,1)-state_info[seed[x]][1]) for x in range(num_seeds)]
        state_outcomes = [0 for x in range(53)]
        electoral_counts = [0,0]
        
        
        for i in range(53):
            shift = 0
            weight_sum = 0
            for x in range(num_seeds):
                shift += correlation_matrix[seed[x]][i]*(seeded_value[x])
                if num_seeds > 1:
                    weight_sum += correlation_matrix[seed[x]][i]
                else:
                    weight_sum = 1
            shift /= weight_sum
            
            state_outcomes[i] = np.random.normal(state_info[i][1]+shift,(state_info[i][2])**0.5,1)

        for i in range(len(state_outcomes)):
            if state_outcomes[i]>0.5:
                electoral_counts[0] += state_info[i][3]
                state_info[i][4]+=1

            elif state_outcomes[i]<0.5:
                electoral_counts[1] += state_info[i][3]
                state_info[i][5]+=1

        if electoral_counts[0]>=270:
            outcome_counts[0]+=1
            electoral_college_outcomes[electoral_counts[0]]+=1
        elif electoral_counts[1]>=270:
            outcome_counts[1]+=1
            electoral_college_outcomes[electoral_counts[0]]+=1
        else:
            outcome_counts[2]+=1
            electoral_college_outcomes[electoral_counts[0]]+=1
  

    for i in state_info:
        i[6] = float(i[4])/num_simulations
        i[7] = float(i[5])/num_simulations
 

In [6]:
def run_adding_after_the_fact(state_info, correlation_matrix, num_simulations):
    for sim in range(num_simulations):
        distance = []
        state_outcomes = [0 for x in range(53)]
        electoral_outcome = [0,0]
        for i in range(53):
            state_outcomes[i] = np.random.normal(state_info[i][1], (state_info[i][2])**0.5,1)
            distance.append((float(state_outcomes[i])-state_info[i][1]))

        shifts = correlation_matrix.dot(distance)

        for i in range(53):

            if state_outcomes[i]+(shifts[i]/np.sum(correlation_matrix[i]))>0.5:
                electoral_outcome[0]+=state_info[i][3]
                state_info[i][4]+=1

            else:
                electoral_outcome[1]+=state_info[i][3]
                state_info[i][5]+=1

        if electoral_outcome[0]>=270:
            outcome_counts[0]+=1
            electoral_college_outcomes[electoral_outcome[0]]+=1
        elif electoral_outcome[1]>=270:
            outcome_counts[1]+=1
            electoral_college_outcomes[electoral_outcome[0]]+=1
        else:
            outcome_counts[2]+=1
            electoral_college_outcomes[electoral_outcome[0]]+=1


    for i in state_info:

        i[6] = float(i[4])/num_simulations
        i[7] = float(i[5])/num_simulations

        print(i)

In [7]:
df=pd.read_csv('AveragedPolls - 2020-10-21.csv',float_precision='round_trip')
pd.set_option("display.precision",15)
demdf = pd.read_csv('Standardized_State_priors.csv',float_precision='round_trip')
pd.set_option("display.precision",15)

In [47]:
demographics = []
state_info = []
outcome_counts = [0,0,0]
electoral_college_outcomes = [x for x in range(539)]
for n in range(53): 
    demographics.append([demdf.iloc[n,demdf.columns.get_loc('whitenonHispanicPercent')],demdf.iloc[n,demdf.columns.get_loc('BlackPercent')],demdf.iloc[n,demdf.columns.get_loc('HispanicPercent')],demdf.iloc[n,demdf.columns.get_loc('nonreligiousPercent')],demdf.iloc[n,demdf.columns.get_loc('medianage')],demdf.iloc[n,demdf.columns.get_loc('Urbanicity')],demdf.iloc[n,demdf.columns.get_loc('Percentcollegedegree')]])
    state_info.append([df.iloc[n,df.columns.get_loc('StateName')], df.iloc[n,df.columns.get_loc('StateMean')], df.iloc[n,df.columns.get_loc('Variance')], df.iloc[n,df.columns.get_loc('ElectoralVotes')], 0,0, 0, 0])
    print(state_info[n])

   

    
    

['Alabama', 0.619323510705329, 0.0053533205785766, 9, 0, 0, 0, 0]
['Alaska', 0.520787233901613, 0.00401347051032681, 3, 0, 0, 0, 0]
['Arizona', 0.481754368867197, 0.0047620412357882, 11, 0, 0, 0, 0]
['Arkansas', 0.66909680263556, 0.00506979446206795, 6, 0, 0, 0, 0]
['California', 0.34176597626492, 0.00238142673823776, 55, 0, 0, 0, 0]
['Colorado', 0.441508063802729, 0.00178425013473564, 9, 0, 0, 0, 0]
['Connecticut', 0.41484746012372, 0.00343173142983656, 7, 0, 0, 0, 0]
['Delaware', 0.443293312927267, 0.00400173142983656, 3, 0, 0, 0, 0]
['District of Columbia', 0.110265808256422, 0.00394173142983656, 3, 0, 0, 0, 0]
['Florida', 0.483662379886877, 0.00456192493440979, 29, 0, 0, 0, 0]
['Georgia', 0.497072195657572, 0.00475020285371373, 16, 0, 0, 0, 0]
['Hawaii', 0.349475442176937, 0.00385221895995009, 4, 0, 0, 0, 0]
['Idaho', 0.608204342235883, 0.00394165735576249, 4, 0, 0, 0, 0]
['Illinois', 0.430976627030397, 0.00329014466368014, 20, 0, 0, 0, 0]
['Indiana', 0.553843622354093, 0.005625444

In [48]:

distances = gen_distances(demographics)
gammas = gamma(distances, 2)
correlation_matrix = rbf_kernel(distances, gammas)


In [49]:
num_simulations = 10000
num_seeds = 6
run_seeding_state_shifts(state_info, correlation_matrix, num_simulations, num_seeds)


In [50]:
for i in state_info:
    print(i)
print("\nModel Predictions:\n")

candidate_1_win_percentage = outcome_counts[0]/num_simulations * 100
candidate_2_win_percentage = outcome_counts[1]/num_simulations * 100

print("Candidate 1 Win%: " + str(candidate_1_win_percentage))   
print("Candidate 2 Win%: " + str(candidate_2_win_percentage))

['Alabama', 0.619323510705329, 0.0053533205785766, 9, 9302, 698, 0.9302, 0.0698]
['Alaska', 0.520787233901613, 0.00401347051032681, 3, 6196, 3804, 0.6196, 0.3804]
['Arizona', 0.481754368867197, 0.0047620412357882, 11, 4088, 5912, 0.4088, 0.5912]
['Arkansas', 0.66909680263556, 0.00506979446206795, 6, 9847, 153, 0.9847, 0.0153]
['California', 0.34176597626492, 0.00238142673823776, 55, 126, 9874, 0.0126, 0.9874]
['Colorado', 0.441508063802729, 0.00178425013473564, 9, 1362, 8638, 0.1362, 0.8638]
['Connecticut', 0.41484746012372, 0.00343173142983656, 7, 944, 9056, 0.0944, 0.9056]
['Delaware', 0.443293312927267, 0.00400173142983656, 3, 2129, 7871, 0.2129, 0.7871]
['District of Columbia', 0.110265808256422, 0.00394173142983656, 3, 0, 10000, 0.0, 1.0]
['Florida', 0.483662379886877, 0.00456192493440979, 29, 4103, 5897, 0.4103, 0.5897]
['Georgia', 0.497072195657572, 0.00475020285371373, 16, 4914, 5086, 0.4914, 0.5086]
['Hawaii', 0.349475442176937, 0.00385221895995009, 4, 276, 9724, 0.0276, 0.972